Изучим набор данных Boston, и с помощью метода k-ближайших соседей решим задачу регрессии - предсказать стоимость жилья, зная различные его характеристики, подробно описанные здесь

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import scale
from sklearn.datasets import load_boston

In [3]:
data = load_boston()

data_X = data.data
data_y = data.target

/home/karen/.local/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing datas

Масштабируем признаки

In [4]:
scaled_data_X = scale(data_X)
scaled_data_X

array([[-0.41978194,  0.28482986, -1.2879095 , ..., -1.45900038,
         0.44105193, -1.0755623 ],
       [-0.41733926, -0.48772236, -0.59338101, ..., -0.30309415,
         0.44105193, -0.49243937],
       [-0.41734159, -0.48772236, -0.59338101, ..., -0.30309415,
         0.39642699, -1.2087274 ],
       ...,
       [-0.41344658, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.98304761],
       [-0.40776407, -0.48772236,  0.11573841, ...,  1.17646583,
         0.4032249 , -0.86530163],
       [-0.41500016, -0.48772236,  0.11573841, ...,  1.17646583,
         0.44105193, -0.66905833]])

Используем KNeighborsRegressor, изменяя p от 1 до 20 с 300 вариантами

In [5]:
k_blocks = 10
p_array = np.linspace(1, 20, 300)
scores = np.empty((300, k_blocks))
for k, p in enumerate(p_array):
    clf = KNeighborsRegressor(n_neighbors=6, weights='distance', p=p)
    scores[k-1] = cross_val_score(clf, scaled_data_X, data_y, cv=k_blocks, 
                                  scoring='neg_mean_squared_error')

In [6]:
sumed_scores = np.sum(scores, axis=1) / k_blocks
max_score = sumed_scores.max()
ind_of_max_score, = np.where(sumed_scores == max_score)
sumed_scores, max_score, p_array[ind_of_max_score + 1]

(array([-28.85472455, -28.60024765, -28.15429858, -28.07228823,
        -27.64078748, -27.48043115, -27.75566893, -27.75753234,
        -27.8705803 , -28.12641183, -28.12295535, -28.06877512,
        -27.72238155, -28.13334412, -28.02583369, -28.00464757,
        -27.60266247, -27.72314496, -27.6348628 , -27.71811087,
        -27.72113969, -27.98119965, -28.02804556, -27.97836085,
        -27.69822883, -27.86250551, -27.82424995, -27.86519031,
        -27.96641045, -28.49693983, -28.52911955, -28.57837557,
        -28.64894066, -28.52662572, -28.77826118, -28.76674603,
        -28.74472046, -28.85070366, -29.12684849, -29.098379  ,
        -29.20044258, -29.15659887, -29.2699474 , -29.20100303,
        -29.26483078, -29.3008271 , -29.32439075, -29.36902595,
        -29.80489072, -29.86403904, -29.78839743, -29.95671995,
        -30.03226671, -30.02830081, -30.08714558, -30.17091927,
        -30.44119751, -30.49743361, -30.74657011, -30.82018692,
        -30.80384048, -30.82537577, -31.

In [8]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


clf = KNeighborsRegressor(n_neighbors=6, weights='distance', p=p_array[ind_of_max_score + 1])
X_train, X_test, y_train, y_test = train_test_split(scaled_data_X, data_y, test_size=0.2, random_state=200)
clf.fit(X_train, y_train)
y_new = clf.predict(X_test)
y = abs(y_test - y_new) * 100 / y_test 
y

array([1.76912718e+01, 7.07484329e+00, 5.83548184e+00, 4.24063516e+01,
       2.63817973e+01, 3.32988752e+01, 2.59081603e+00, 1.59794672e+00,
       4.51533024e+00, 1.52555130e+01, 8.05497502e+01, 6.71836607e+00,
       1.08275285e+01, 2.83849483e+00, 1.58201794e+01, 1.34870291e+01,
       3.10054375e+00, 1.46620928e+01, 4.49073501e+01, 1.78313998e+01,
       3.06837997e+01, 3.75214205e-01, 3.03133106e+01, 2.19985139e+01,
       4.05784747e+00, 7.34804586e-02, 5.83958808e-01, 1.53399322e+01,
       5.53207073e+00, 4.27628468e+00, 2.37898801e+01, 3.87480065e-01,
       1.04883092e+01, 3.54315210e+01, 3.74361371e+00, 5.65192705e+01,
       1.67294839e-01, 9.51767299e+00, 3.19652855e+00, 4.73626724e+00,
       2.42706825e+01, 7.93303537e+00, 4.61387178e+00, 2.87554144e+00,
       5.04291206e+01, 4.92179625e+01, 3.55698365e+00, 1.66104697e+01,
       3.07174661e+01, 2.01123374e+01, 3.06676059e+00, 4.29587283e+01,
       3.75908713e+01, 2.00263085e+00, 4.00064611e+01, 4.24696185e+00,
      